In [1]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
from collections import Counter

In [2]:
news = pd.read_csv("estadao_noticias_eleicao.csv", encoding="utf-8")
news = news.replace(np.nan, '', regex = True)
news.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


In [3]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

### Escreva uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).

Essa parte do código foi disponibilizado nesse link https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb

In [4]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

In [6]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

In [7]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [8]:
matrix, vocab = co_occurrence_matrix(tokens)

In [9]:
consultable_matrix = matrix.tocsr()

In [10]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

### Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.

Primeiramente inverter o vocab, de forma que as chaves sejam os números e os valores as palavras, para facilitara na busca pelo top 3 de co-ocorrência.

In [12]:
i_vocab = {vocab[key]: key for key in vocab}

Feito isso na função top3 vamos utilizar a consultable_matrix, que contém as quantidades de co-ocorrências de cada palavra com todas as outras do corpus, e pegar a linha dessa matrix correspondente as co-ocorrências da palavra que queremos transformar em um array, onde cada posição está relacionada com a chave do i_vocab, por exemplo, a posição 1 do array co_ocorr_words, contém a quantidade de co-ocorrências de word com a palavra retornada de i_vocab[1] 

In [13]:
def top3(word):
    co_ocorr_words = np.reshape(consultable_matrix[vocab[word]].toarray(), -1)
    
    dic = {}
    
    for i in range(len(co_ocorr_words)):
        dic[i_vocab[i]] = co_ocorr_words[i]
        
    return sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:3]

### Expanda a consulta original

Agora a nova consulta será composta pela consulta original mais as palavras retornadas do top3

In [15]:
def expand(word):
    query = []
    query.append(word)
    query.extend(i[0] for i in top3(word))
    
    return " ".join(i for i in query)

In [18]:
def gera_tokens(df):
    indexes = defaultdict(set)
    for index,row in df.iterrows():
        if isinstance(row['titulo'], str):
            titulo_tokens = (word.lower() for word in (nltk.word_tokenize(row['titulo'])))
        if isinstance(row['subTitulo'], str):
            subTitulo_tokens = (word.lower() for word in (nltk.word_tokenize(row['subTitulo'])))
        if isinstance(row['conteudo'], str):
            conteudo_tokens = (word.lower() for word in (nltk.word_tokenize(row['conteudo'])))
            a = Counter(word.lower() for word in (nltk.word_tokenize(row['conteudo']))) #TF (term frequency)
        #if isinstance(row['url'], str):
            #url_tokens = (word.lower() for word in (nl.word_tokenize(row['url'])))
        
        
        for token in titulo_tokens:
            indexes[token].add( (int(row['idNoticia']), a[token]) )
                
        for token in subTitulo_tokens:
            indexes[token].add( (int(row['idNoticia']), a[token]) )

        for token in conteudo_tokens:
            indexes[token].add( ( int(row['idNoticia']), a[token] ) )
            
        #for token in url_tokens:
            #indexes[token].add( (int(row['idNoticia']), a[token]) )
        
    return indexes

In [19]:
dic = gera_tokens(news)

In [22]:
def search(word):
    
    docs = dic[word]
    
    return [doc[0] for doc in docs]

Escolhendo os termos dilma, aécio e discriminalização para serem a consulta

##### Quais os termos retornados para a expansão de cada consulta?

In [28]:
print(expand('dilma'))
print(expand('aécio'))
print(expand('descriminalização'))

dilma rousseff disse é
aécio neves disse é
descriminalização aborto maconha drogas


##### Você acha que esses termos são de fato relacionados com a consulta original?

Com exceção das palavras 'disse' e 'é', os demais termos tem relação com os termos, já que rousseff e neves são sobrenomes de dilma e aécio, e aborto, maconha e drogas são coisas bem ligadas ao assunto de decriminalização.

##### Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?

Em relação as consultas pelos termos 'dilma', 'aécio', a consulta expandida será pior que a consulta simples, uma vez que muitos documentos que contém os termos 'disse' e 'é' não teram relação nenhuma com os termos 'dilma' e 'aécio'

Já no outro caso, a consulta expandida para discriminalização será melhor, já que o usuário poderia querer se informar da discriminalização de algo especifico.

##### A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

Se forem recuperados muitos documentos e poucos deles forem relevantes:
O recall aumenta e o precision diminui

Se a quantidade de documentos relevantes for proxima ao número de documentos recuperados:
O precision aumenta e o recall diminui

Em geral, como a expansão de consulta recupera muitos documentos, é mais adequada para melhorar o recall